In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import cv2
import tqdm

# Using seaborn's style
plt.style.use('ggplot')
width = 'thesis'

tex_fonts = {
    # Use LaTeX to write all text
    "text.usetex": True,
    "font.family": "serif",
    # Use 10pt font in plots, to match 10pt font in document
    "axes.labelsize": 10,
    "font.size": 10,
    # Make the legend/label fonts a little smaller
    "legend.fontsize": 8,
    "xtick.labelsize": 8,
    "ytick.labelsize": 8,
    # Remove grid lines
    "axes.grid": False,
    # Show the ticks for the x and y axes
    "axes.spines.bottom": False,
    "axes.spines.left": False,
    "xtick.major.size" : 6,
    "ytick.major.size" : 6, 
    "xtick.minor.size" : 3, 
    "ytick.minor.size" : 3,
    "xtick.major.width" : 1,
    "ytick.major.width" : 1,
    "xtick.minor.width" : 1,
    "ytick.minor.width" : 1,
    # Outline the plot area
    "axes.spines.top": False,
    "axes.spines.right": False,
}

plt.rcParams.update(tex_fonts)

In [2]:
wd = "C:\\Users\\simulationsFFlab\\Desktop\\Simulations_Thesis"
os.chdir(wd)

In [3]:
# Search the current directory for folders with the word simulation_2023 in them
# These folders contain the data from the simulations
folders = [f for f in os.listdir() if 'simulation_2023' in f]

In [4]:
Dms = [
    "DM_0.01",
    "DM_0.05",
    "DM_0.1",
    "DM_0.25",
    "DM_0.5",
    "DM_0.75",
    "DM_1.0",
    "DM_3.0",
    "DM_5.0",
    "DM_10.0",
    ]
De_vals = [0.01, 0.05, 0.1, 0.5, 1.0, 5.0, 10.0, 25.0, 50.0, 75.0]
Des = ["DE_{}".format(i) for i in De_vals]


In [5]:
# Check whether there is a folder names IMAGES_THESIS in the current directory
# If not, create one
if not os.path.exists("IMAGES_THESIS"):
    os.mkdir("IMAGES_THESIS")

In [18]:
image_array = np.zeros((256, 256, 3))
# Loop over the folders (use tqdm to show progress bar)
for folder in tqdm.tqdm(folders):
    # Change the working directory to the folder
    os.chdir(wd + "\\" + folder)
    # List the files in the folder if the files end with .npz
    files = [f for f in os.listdir() if f.endswith('.npz')]
    # Check whether there is a folder with the name of the folder in IMAGES_THESIS
    # If not, create one
    if not os.path.exists(wd + "\\IMAGES_THESIS\\" + folder):
        os.mkdir(wd + "\\IMAGES_THESIS\\" + folder)
    # Sort the files by the third integer after splitting by underscores
    files.sort(key=lambda f: int((f.split('_')[2]).split('.')[0]))
    # For all .npz files in the folder, loop over them and load the rfp anf gfp arrays
    for file in files:
        # Load the arrays
        data = np.load(file)
        rfp = data['rfp']
        gfp = data['gfp']
        # Normalize the arrays between the minimum of the two and the maximum of the two
        # This is done to make sure that the colors are consistent between the images
        rfp = (rfp - np.min([np.min(rfp), np.min(gfp)])) / (np.max([np.max(rfp), np.max(gfp)]) - np.min([np.min(rfp), np.min(gfp)]))
        gfp = (gfp - np.min([np.min(rfp), np.min(gfp)])) / (np.max([np.max(rfp), np.max(gfp)]) - np.min([np.min(rfp), np.min(gfp)]))
        # Normalize again between 0 and 1
        rfp = rfp / np.max(rfp)
        gfp = gfp / np.max(gfp)
        # Create the image array (remember that cv2 uses BGR instead of RGB)
        image_array[:, :, 0] = 0
        image_array[:, :, 1] = gfp
        image_array[:, :, 2] = rfp
        # Convert the array to uint8
        image_array = (image_array * 255).astype(np.uint8)
        # Save the image
        cv2.imwrite(wd + "\\IMAGES_THESIS\\" + folder + "\\" + file.split('.')[0] + ".png", image_array)
        # Reset the image array
        image_array = np.zeros((256, 256, 3))

    # Create a video from the images
    # Change the working directory to the folder with the images
    os.chdir(wd + "\\IMAGES_THESIS\\" + folder)
    # List the files in the folder
    files = [f for f in os.listdir() if f.endswith('.png')]
    # Sort the files by the third integer after splitting by underscores
    files.sort(key=lambda f: int((f.split('_')[2]).split('.')[0]))
    # Create a video writer, set the codec to mp4v, the fps to 10 and the size to 256x256
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(folder + '.mp4', fourcc, 10.0, (256, 256))
    # Loop over the files
    for file in files:
        # Read the image
        img = cv2.imread(file)
        # Write the image to the video
        out.write(img)
    # Change the working directory back to the main directory
    os.chdir(wd)
       

  0%|          | 0/500 [00:00<?, ?it/s]C:\Users\simulationsFFlab\AppData\Local\Temp\ipykernel_15780\1533413815.py:25: RuntimeWarning: invalid value encountered in divide
  rfp = rfp / np.max(rfp)
C:\Users\simulationsFFlab\AppData\Local\Temp\ipykernel_15780\1533413815.py:32: RuntimeWarning: invalid value encountered in cast
  image_array = (image_array * 255).astype(np.uint8)
100%|██████████| 500/500 [25:14<00:00,  3.03s/it]


In [6]:
# Filter out the folders that contain "DM_1.0" and "DE_10.0"
# These folders contain the data from the simulations
folders = [f for f in folders if "DM_1.0" in f and "DE_10.0" in f]

In [8]:
# Loop over the folders (use tqdm to show progress bar)
for folder in tqdm.tqdm(folders):
    # Change the working directory to the folder
    os.chdir(wd + "\\" + folder)
    # List the files in the folder if the files end with .npz
    files = [f for f in os.listdir() if f.endswith('.npz')]
    # Check whether there is a folder with the name of the folder in IMAGES_THESIS
    # If not, create one
    if not os.path.exists(wd + "\\IMAGES_THESIS\\" + folder):
        os.mkdir(wd + "\\IMAGES_THESIS\\" + folder)
    # Sort the files by the third integer after splitting by underscores
    files.sort(key=lambda f: int((f.split('_')[2]).split('.')[0]))
    # Create an empty array to store the data for the rpf and gfp
    full_data = np.zeros((256, 256, 2, len(files)))
    # For all .npz files in the folder, loop over them and load the rfp anf gfp arrays
    for file in files:
        # Load the arrays
        data = np.load(file)
        rfp = data['rfp']
        gfp = data['gfp']
        # Add the arrays to the full_data array
        full_data[:, :, 0, files.index(file)] = rfp
        full_data[:, :, 1, files.index(file)] = gfp
    # Consider that each of the points in the array corresponds to a cell expressing gfp and rfp
    # We have to plot the dynamics of the gfp and rfp expression for each of the cells
    # To do this, we have to loop over the first two dimensions of the array
    # Create a figure with one plot, the rfp and gfp will be plotted on the same plot
    fig, ax = plt.subplots(figsize=(6, 3), dpi=300)
    # Loop over the first two dimensions of the array
    for i in range(full_data.shape[0]):
        for j in range(full_data.shape[1]):
            # Plot the data
            ax.plot(full_data[i, j, 0, :], color='red', alpha=0.1)
            ax.plot(full_data[i, j, 1, :], color='green', alpha=0.1)
    # Set the x and y labels
    ax.set_xlabel('Tiempo (min)')
    ax.set_ylabel('Concentración de Reportero ($\mu M$)')
    # Set the title
    ax.set_title('Dinámica de expresión del sistema bidimensional')
    # Save the figure
    plt.savefig(wd + "\\IMAGES_THESIS\\" + folder + "\\" + folder + "_EXPRESSION_DYNAMICS.png", dpi=300)
    # Close the figure
    plt.close(fig)

  0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 5/5 [14:02<00:00, 168.45s/it]
